In [22]:
import pickle
import pickle
import numpy as np
#import utils
import pandas as pd
import time

class HyperParas:
    def __init__(self):
        #with open('raw_gtap_dataset/regions_list.pkl', 'rb') as f:
        reg_one =  pd.read_excel('Downloads/GTAP/GDX11a17/GSDFDAT_reg.xlsx', header=None)[0].tolist()
        self.regions_list = [activity.lower() for activity in reg_one]
        #with open('raw_gtap_dataset/activities_list.pkl', 'rb') as f:
        act =  pd.read_excel('Downloads/GTAP/GDX11a17/GSDFDAT_act.xlsx',header=None)[0].tolist()
        # Transform all items to lowercase
        self.activities_list = [activity.lower() for activity in act]
        self.num_regions = len(self.regions_list)
        self.num_commodities = len(self.activities_list)
        self.num_items = self.num_regions * self.num_commodities

In [23]:
#from hyperparas import HyperParas
hyper = HyperParas()

In [3]:
### Utils

In [24]:
import pandas as pd
import numpy as np
#from hyperparas import HyperParas
import time

In [25]:
def df_to_COMM_REG_dict(df, key_list):
    # transfer df(obtained using txt files in original gtap dataset) to dict
    dict_ = {}
    for i, row in df.iterrows():
        dict_key = tuple([row[key_] for key_ in key_list])
        dict_[dict_key] = row['VALUE']
    return dict_


def txt_to_COMM_REG_dict(txt_path, key_list):
    # transfer txt files in original gtap dataset to dict
    return df_to_COMM_REG_dict(pd.read_table(txt_path, delimiter=','), key_list)


def get_item_index(comm, reg, hyper: HyperParas):
    comm_index, reg_index = hyper.activities_list.index(comm.lower()), hyper.regions_list.index(reg.lower())
    return reg_index * hyper.num_commodities + comm_index


def raw_dict_to_matrix(raw_dict, matrix_type, key_mapping, hyper: HyperParas):
    if matrix_type == 'Z':
        Z = np.zeros((hyper.num_items, hyper.num_items))
        for key_info in raw_dict:
            REG_SOU, COMM_SOU, REG_TAR, COMM_TAR = key_info[key_mapping['REG_SOU_index_in_key']], \
                                                   key_info[key_mapping['COMM_SOU_index_in_key']], \
                                                   key_info[key_mapping['REG_TAR_index_in_key']], \
                                                   key_info[key_mapping['COMM_TAR_index_in_key']]
            loc_row, loc_col = get_item_index(COMM_SOU, REG_SOU, hyper), \
                               get_item_index(COMM_TAR, REG_TAR, hyper)
            Z[loc_row, loc_col] = raw_dict[key_info]
        return Z
    elif matrix_type == 'F':
        F = np.zeros((hyper.num_items, hyper.num_regions))
        for key_info in raw_dict:
            REG_SOU, COMM_SOU, REG_TAR = key_info[key_mapping['REG_SOU_index_in_key']], \
                                         key_info[key_mapping['COMM_SOU_index_in_key']], \
                                         key_info[key_mapping['REG_TAR_index_in_key']]
            loc_row, loc_col = get_item_index(COMM_SOU, REG_SOU, hyper), \
                               hyper.regions_list.index(REG_TAR.lower())
            F[loc_row, loc_col] = raw_dict[key_info]
        return F
    elif matrix_type == 'comm_to_firm':
        comm_to_firm = np.zeros((hyper.num_commodities, hyper.num_items))
        for key_info in raw_dict:
            COMM_SOU, REG_TAR, COMM_TAR = key_info[key_mapping['COMM_SOU_index_in_key']], \
                                          key_info[key_mapping['REG_TAR_index_in_key']], \
                                          key_info[key_mapping['COMM_TAR_index_in_key']]
            loc_row, loc_col = hyper.activities_list.index(COMM_SOU.lower()), get_item_index(COMM_TAR, REG_TAR, hyper)
            comm_to_firm[loc_row, loc_col] = raw_dict[key_info]
        return comm_to_firm
    elif matrix_type == 'comm_to_region_final':
        comm_to_region_final = np.zeros((hyper.num_commodities, hyper.num_regions))
        for key_info in raw_dict:
            COMM_SOU, REG_TAR = key_info[key_mapping['COMM_SOU_index_in_key']], \
                                key_info[key_mapping['REG_TAR_index_in_key']]
            loc_row, loc_col = hyper.activities_list.index(COMM_SOU.lower()), hyper.regions_list.index(REG_TAR.lower())
            comm_to_region_final[loc_row, loc_col] = raw_dict[key_info]
        return comm_to_region_final
    elif matrix_type == 'firm_to_region_final':
        firm_to_region_final = np.zeros((hyper.num_items, hyper.num_regions))
        for key_info in raw_dict:
            COMM_SOU, REG_SOU, REG_TAR = key_info[key_mapping['COMM_SOU_index_in_key']], \
                                         key_info[key_mapping['REG_SOU_index_in_key']], \
                                         key_info[key_mapping['REG_TAR_index_in_key']]
            loc_row, loc_col = get_item_index(COMM_SOU, REG_SOU, hyper), hyper.regions_list.index(REG_TAR.lower())
            firm_to_region_final[loc_row, loc_col] = raw_dict[key_info]
        return firm_to_region_final
    elif matrix_type == 'va_all_factors':
        va_all_factors = np.zeros(hyper.num_items)
        for key_info in raw_dict:
            COMM, REG = key_info[key_mapping['COMM_index_in_key']], \
                        key_info[key_mapping['REG_index_in_key']]
            loc = get_item_index(COMM, REG, hyper)
            va_all_factors[loc] += raw_dict[key_info]
        return va_all_factors


def row_sum_n_col(array_, n_col):
    # row sum: for each row, [sum of columns: 0~n_col-1, sum of columns: n_col~2*n_col-1, ...]
    return np.sum(np.reshape(array_, (-1, int(np.shape(array_)[1] / n_col), n_col)), axis=2)


def col_sum_each_n_row(array_, n_row):
    # col sum: for each column, [sum of rows: 0,n_row, 2*n_row...;
    #                            sum of rows: 1,n_row+1, 2*n_row+1...;
    #                            ...]
    return np.sum(np.reshape(array_, (-1, n_row, int(np.shape(array_)[1]))), axis=0)


def recover_international_Z_F(imports_firms_to_region, imports_comm_to_firm, imports_comm_to_region_final,
                              exports_firm_to_region, hyper):
    imports_comm_to_region = col_sum_each_n_row(imports_firms_to_region,
                                                hyper.num_commodities)
    product_distribution_firm = imports_comm_to_firm / np.repeat(imports_comm_to_region, hyper.num_commodities, axis=1)
    product_distribution_final = imports_comm_to_region_final / imports_comm_to_region

    return np.repeat(exports_firm_to_region, hyper.num_commodities, axis=1) * np.tile(product_distribution_firm,
                                                                                      (hyper.num_regions, 1)), \
           exports_firm_to_region * np.tile(product_distribution_final, (hyper.num_regions, 1))


def RAS_matrix_balancing(A, u, v):
    """
    balance matrix based on the RAS approach
    :param A: original matrix
    :param u: targeted row sum
    :param v: targeted col sum
    :return: X: balanced matrix
    """
    X = A.copy()
    print('------balancing.....------')
    for i in range(100):
        b = time.time()
        R = u / np.sum(X, axis=1)
        X = X * R[:, None]
        S = v / np.sum(X, axis=0)
        X = X * S[None, :]
        print(i, time.time() - b)
    return X


def MRIOT_adjust(Z_, VA_, F_, num_comm_):
    """
    :param Z_: shape: num_item, num_item
    :param VA_: shape: num_item
    :param F_: shape: num_item, num_reg
    :param num_reg_:
    :param num_comm_:
    :return:
    """
    # keep constant
    Y_without_VA = np.sum(Z_, axis=0)
    VA_by_country = row_sum_n_col(np.reshape(VA_, (1, -1)), num_comm_).flatten()

    # targeted row sum
    U = Y_without_VA + VA_
    # targeted col sum
    V = np.concatenate((Y_without_VA, VA_by_country))
    # Z_F_MRIOT: not including VA in balancing
    Z_F_MRIOT = np.concatenate((Z_, F_), axis=1)

    Z_F_MRIOT_adjusted = RAS_matrix_balancing(Z_F_MRIOT, U, V)
    return Z_F_MRIOT_adjusted


if __name__ == '__main__':
    # sample check
    num_regions, num_commodities = 3, 4
    num_items = num_regions * num_commodities
    Z = np.random.randint(1, 4, (num_items, num_items))
    VA = np.random.randint(1, 4, num_items)
    F = np.random.randint(1, 4, (num_items, num_regions))
    mriot = MRIOT_adjust(Z, VA, F, num_commodities)

    # condition check
    Y_without_VA = np.sum(Z, axis=0)
    VA_by_country = row_sum_n_col(np.reshape(VA, (1, -1)), num_commodities).flatten()
    Y_init = Y_without_VA + VA

    print('----- init ----------------')
    mriot_init = np.concatenate((Z, F), axis=1)
    mriot_row = np.sum(mriot_init, axis=1)
    print(mriot_row, Y_init)

    mriot_col = np.sum(mriot_init, axis=0)
    print(mriot_col[:num_items], Y_init)
    print(mriot_col[num_items:], VA_by_country)

    print('----- balanced ----------------')
    mriot_balance_row = np.sum(mriot, axis=1)
    print(mriot_balance_row, Y_init)

    mriot_balance_col = np.sum(mriot, axis=0)
    print(mriot_balance_col[:num_items], Y_init)
    print(mriot_balance_col[num_items:], VA_by_country)

------balancing.....------
0 0.0009968280792236328
1 0.0
2 0.0
3 0.0
4 0.0
5 0.0
6 0.0
7 0.0
8 0.0
9 0.0
10 0.0
11 0.0
12 0.0
13 0.0
14 0.0
15 0.0
16 0.0
17 0.0
18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 0.0
30 0.0
31 0.0
32 0.0
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.0
41 0.0
42 0.0009982585906982422
43 0.0
44 0.0
45 0.0
46 0.0
47 0.0
48 0.0
49 0.0
50 0.0
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.0
57 0.0
58 0.0
59 0.0
60 0.0
61 0.0
62 0.0
63 0.0
64 0.0
65 0.0
66 0.0
67 0.0
68 0.0
69 0.0
70 0.0
71 0.0
72 0.0
73 0.0
74 0.0
75 0.0
76 0.0
77 0.0
78 0.0
79 0.0
80 0.0
81 0.0
82 0.0
83 0.0
84 0.0
85 0.0
86 0.0
87 0.0
88 0.0
89 0.0
90 0.0
91 0.0
92 0.0
93 0.0
94 0.0
95 0.0
96 0.0
97 0.0
98 0.0
99 0.0
----- init ----------------
[26 29 27 28 25 30 31 30 28 33 35 25] [28 28 20 23 23 22 23 26 20 28 26 30]
[25 26 19 21 21 20 22 24 18 25 24 29] [28 28 20 23 23 22 23 26 20 28 26 30]
[26 26 21] [8 7 8]
----- balanced ----------------
[28. 28. 20. 23.

In [26]:
def get_Z_domestic():
    ### get info for domestic Z
    # lowercase_uppercase: code in gtap v7_code in gtap v10
    # purchase: from COMM_SOU in REG to COMM_TAR in REG
    #column_names = ['COMM_SOU', 'COMM_TAR', 'REG', 'VALUE']

    # Load the Excel file with the specified column names
    #vdfm_VDFB_dict = pd.read_excel('Downloads/GTAP/GDX11a17/GSDFDAT_VDFB.xlsx', header=None, names=column_names)

    vdfm_VDFB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VDFB.txt', ['COMM_SOU', 'COMM_TAR', 'REG'])
    # construct domestic Z
    Z_domestic = raw_dict_to_matrix(vdfm_VDFB_dict, 'Z', {'REG_SOU_index_in_key': 2,
                                                                'COMM_SOU_index_in_key': 0,
                                                                'REG_TAR_index_in_key': 2,
                                                                'COMM_TAR_index_in_key': 1}, hyper)
    return Z_domestic


def get_F_domestic():
    ## get info for domestic y
    # final domestic household: from COMM in REG to Final in REG
    vdpm_VDPB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VDPB.txt', ['COMM', 'REG'])
    domestic_firm_to_region_final_house = raw_dict_to_matrix(vdpm_VDPB_dict, 'firm_to_region_final',
                                                                   {'COMM_SOU_index_in_key': 0,
                                                                    'REG_SOU_index_in_key': 1,
                                                                    'REG_TAR_index_in_key': 1}, hyper)
    # final domestic gov: from COMM in REG to Final in REG
    vdgm_VDGB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VDGB.txt', ['COMM', 'REG'])
    domestic_firm_to_region_final_gov = raw_dict_to_matrix(vdgm_VDGB_dict, 'firm_to_region_final',
                                                                 {'COMM_SOU_index_in_key': 0,
                                                                  'REG_SOU_index_in_key': 1,
                                                                  'REG_TAR_index_in_key': 1}, hyper)

    # final domestic inv: from COMM in REG to Final in REG
    vdkm_VDIB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VDIB.txt', ['COMM', 'REG'])
    domestic_firm_to_region_final_inv = raw_dict_to_matrix(vdkm_VDIB_dict, 'firm_to_region_final',
                                                                 {'COMM_SOU_index_in_key': 0,
                                                                  'REG_SOU_index_in_key': 1,
                                                                  'REG_TAR_index_in_key': 1}, hyper)
    # final domestic vst: from COMM in REG to Final in REG
    vst_VST_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VST.txt', ['COMM', 'REG'])
    domestic_firm_to_region_final_vst = raw_dict_to_matrix(vst_VST_dict, 'firm_to_region_final',
                                                                 {'COMM_SOU_index_in_key': 0,
                                                                  'REG_SOU_index_in_key': 1,
                                                                  'REG_TAR_index_in_key': 1}, hyper)

    # final domestic all
    F_domestic = domestic_firm_to_region_final_house + domestic_firm_to_region_final_gov + \
                 domestic_firm_to_region_final_inv + domestic_firm_to_region_final_vst
    return F_domestic


def get_international_Z_F():
    ### get info for international Z
    # exports: from COMM in REG_SOU to REG_TAR
    vxmd_VXSB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VXSB.txt', ['COMM', 'REG_SOU', 'REG_TAR'])
    exports_firm_to_region = raw_dict_to_matrix(vxmd_VXSB_dict, 'F', {'REG_SOU_index_in_key': 1,
                                                                           'COMM_SOU_index_in_key': 0,
                                                                            'REG_TAR_index_in_key': 2}, hyper)
    # imports: COMM_SOU to COMM_TAR in REG
    vifm_VMFB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VMFB.txt', ['COMM_SOU', 'COMM_TAR', 'REG'])
    imports_comm_to_firm = raw_dict_to_matrix(vifm_VMFB_dict, 'comm_to_firm', {'COMM_SOU_index_in_key': 0,
                                                                                     'COMM_TAR_index_in_key': 1,
                                                                                     'REG_TAR_index_in_key': 2}, hyper)
    # imports: from COMM in REG_SOU to REG_TAR
    vims_VMSB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VMSB.txt',  ['COMM', 'REG_SOU', 'REG_TAR'])
    imports_firms_to_region = raw_dict_to_matrix(vims_VMSB_dict, 'F',
                                                      {'COMM_SOU_index_in_key': 0,
                                                       'REG_SOU_index_in_key': 1,
                                                       'REG_TAR_index_in_key': 2}, hyper)

    # final imports household: COMM to Final in REG
    vipm_VMPB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VMPB.txt', ['COMM', 'REG'])
    imports_comm_to_region_final_house = raw_dict_to_matrix(vipm_VMPB_dict, 'comm_to_region_final',
                                                                  {'COMM_SOU_index_in_key': 0,
                                                                   'REG_TAR_index_in_key': 1}, hyper)
    ## get infor for international y
    # final imports gov: COMM to Final in REG
    vigm_VMGB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VMGB.txt', ['COMM', 'REG'])
    imports_comm_to_region_final_gov = raw_dict_to_matrix(vigm_VMGB_dict, 'comm_to_region_final',
                                                                {'COMM_SOU_index_in_key': 0,
                                                                 'REG_TAR_index_in_key': 1}, hyper)

    # final imports inv: COMM to Final in REG
    vikm_VMIB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/VMIB.txt', ['COMM', 'REG'])
    imports_comm_to_region_final_inv = raw_dict_to_matrix(vikm_VMIB_dict, 'comm_to_region_final',
                                                                {'COMM_SOU_index_in_key': 0,
                                                                 'REG_TAR_index_in_key': 1}, hyper)
    # total imports to final
    imports_comm_to_region_final = \
        imports_comm_to_region_final_house + imports_comm_to_region_final_gov + imports_comm_to_region_final_inv

    # construct international Z & y
    Z_international, F_international = recover_international_Z_F(imports_firms_to_region, imports_comm_to_firm,
                                                                       imports_comm_to_region_final,
                                                                       exports_firm_to_region, 
                                                                 hyper)
    return Z_international, F_international


def get_value_add_firm():
    # get info for value added
    vfm_EVFB_dict = txt_to_COMM_REG_dict('Downloads/GTAP/GDX11a17/EVFB.txt', ['FAC', 'COMM', 'REG'])
    factor_purchase_firms_vfm = raw_dict_to_matrix(vfm_EVFB_dict, 'va_all_factors',
                                                         {'COMM_index_in_key': 1,
                                                          'REG_index_in_key': 2}, hyper)
    value_added_firm = factor_purchase_firms_vfm
    return value_added_firm


def get_raw_MRIOT():
    # get info for value-added cal
    print('------------ Constructing domestic Z... --------------')
    b = time.time()
    Z_domestic = get_Z_domestic()
    print('------------ Done ({}s) --------------'.format(time.time()-b))

    print('------------ Constructing domestic F... --------------')
    b = time.time()
    F_domestic = get_F_domestic()
    print('------------ Done ({}s) --------------'.format(time.time()-b))

    print('------------ Constructing international Z and F... --------------')
    b = time.time()
    Z_international, F_international = get_international_Z_F()
    print('------------ Done ({}s) --------------'.format(time.time()-b))

    Z = Z_domestic + Z_international
    F = F_domestic + F_international

    print('------------ Constructing VA... --------------')
    b = time.time()
    VA = get_value_add_firm()
    print('------------ Done ({}s) --------------'.format(time.time()-b))

    print('------------ Saving raw Z, F, VA... --------------')
    b = time.time()
    #with open('processed_gtap/Z_init.pkl', 'wb') as f:
    #    pickle.dump(Z, f)
    #with open('processed_gtap/F_init.pkl', 'wb') as f:
    #    pickle.dump(F, f)
    #with open('processed_gtap/VA_init.pkl', 'wb') as f:
    #    pickle.dump(VA, f)
    print('------------ Done ({}s) --------------'.format(time.time()-b))
    
    return Z, F, VA


#  read raw data
Z, F, VA = get_raw_MRIOT()


------------ Constructing domestic Z... --------------
------------ Done (44.904350996017456s) --------------
------------ Constructing domestic F... --------------
------------ Done (1.7834410667419434s) --------------
------------ Constructing international Z and F... --------------
------------ Done (294.742933511734s) --------------
------------ Constructing VA... --------------
------------ Done (3.685162305831909s) --------------
------------ Saving raw Z, F, VA... --------------
------------ Done (0.0s) --------------


In [27]:
adjusted_MRIOT_ZF_part = MRIOT_adjust(Z, VA, F, int(len(VA) / np.shape(F)[1]))

------balancing.....------
0 1.002790927886963
1 1.0292465686798096
2 0.9395320415496826
3 0.9604270458221436
4 0.9376301765441895
5 0.9108145236968994
6 0.9384961128234863
7 0.8666434288024902
8 0.8756568431854248
9 0.8388152122497559
10 0.8447840213775635
11 0.823798656463623
12 0.8227677345275879
13 0.8286607265472412
14 0.7958662509918213
15 0.8217422962188721
16 0.8317751884460449
17 0.8307766914367676
18 0.8427951335906982
19 0.782853364944458
20 0.8308169841766357
21 0.8277885913848877
22 0.8268280029296875
23 0.8138227462768555
24 0.8389337062835693
25 0.7928931713104248
26 0.8167924880981445
27 0.8207519054412842
28 0.8288393020629883
29 0.8397543430328369
30 0.8277907371520996
31 0.812828540802002
32 0.8178141117095947
33 0.8227310180664062
34 0.815873384475708
35 0.8347103595733643
36 0.7938740253448486
37 0.8467981815338135
38 0.8298239707946777
39 0.8167510032653809
40 0.8378138542175293
41 0.840752363204956
42 0.7918856143951416
43 0.8237960338592529
44 0.8138229846954346

In [28]:
Y_without_VA = np.sum(Z, axis=0)
VA_by_country = row_sum_n_col(np.reshape(VA, (1, -1)), 65).flatten()
Y_init = Y_without_VA + VA

mriot_balance_row = np.sum(adjusted_MRIOT_ZF_part, axis=1)
mriot_balance_col = np.sum(adjusted_MRIOT_ZF_part, axis=0)

In [ ]:
####

In [29]:
MRIOT = np.concatenate((adjusted_MRIOT_ZF_part,
                        np.concatenate([np.reshape(VA, (1, -1)),
                                        np.ones((1, hyper.num_regions)) * np.nan], axis=1)),
                       axis=0)
df = pd.DataFrame(MRIOT)

In [30]:
indexes = []
for region in hyper.regions_list:
    for act in hyper.activities_list:
        indexes.append(region + '_' + act)

# Setting new index (rows)
df = df.set_axis(indexes + ['VA'], axis=0)

# Setting new columns
df = df.set_axis(indexes +  ['y_' + region for region in hyper.regions_list], axis=1)

In [31]:
df

,aus_pdr,aus_wht,aus_gro,aus_v_f,aus_osd,aus_c_b,aus_pfb,aus_ocr,aus_ctl,aus_oap,...,y_uga,y_zmb,y_zwe,y_xec,y_bwa,y_swz,y_nam,y_zaf,y_xsc,y_xtw
aus_pdr,3.563641e-01,5.006335e-01,2.419844e-01,1.775985e-01,2.211215e-01,1.339100e-03,1.763873e-01,2.684434e-01,1.362141e+00,2.921292e-01,...,4.998362e-08,1.862827e-07,3.232549e-07,2.232570e-07,3.005088e-13,1.370166e-11,7.026693e-11,1.824943e-07,2.094865e-11,7.121225e-11
aus_wht,1.761312e+00,6.080293e+01,2.597075e+01,6.270146e-02,2.324497e+01,1.967180e-03,5.866347e-01,4.234976e-01,7.321676e+01,1.573446e+01,...,1.020973e+00,3.011060e-06,4.503865e-07,2.996681e-06,1.690244e-10,2.502827e-10,1.106031e-11,9.510627e-01,7.890919e-10,9.209200e-09
aus_gro,9.335614e-01,2.761221e+01,1.655588e+01,3.206441e-02,1.230199e+01,1.490292e-02,2.498806e-02,1.988038e-02,4.036649e+01,4.158333e+01,...,2.093331e-01,2.016747e-04,1.689877e-05,5.773307e-06,1.054143e-02,4.251847e-09,3.572445e-09,3.366105e-01,6.812551e-09,1.132522e-08
aus_v_f,6.943234e-03,4.459760e-02,4.401031e-02,1.101124e+02,5.583700e-03,1.817646e+01,2.239915e+01,1.145682e+01,1.455062e+00,7.984388e-01,...,1.064037e-03,1.309770e-03,3.417663e-04,8.915960e-02,1.887010e-02,1.367732e-08,1.021416e-07,3.768329e-01,1.457562e-08,9.380428e-07
aus_osd,1.879592e+00,5.560353e+01,2.767140e+01,1.327213e+01,3.490363e+01,2.234366e+00,3.826910e+00,2.059659e+00,7.825267e+01,1.685273e+01,...,4.150642e-08,1.268111e-06,7.611718e-07,1.249232e-07,1.344371e-10,1.552592e-12,3.561753e-13,9.008342e-01,2.753241e-12,3.779610e-10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xtw_osg,7.003890e-10,1.201421e-09,1.724222e-09,1.503366e-07,4.228976e-09,2.882576e-08,8.842323e-08,7.060723e-08,7.882096e-10,2.267509e-09,...,3.057952e-05,8.440006e-05,2.761712e-05,2.438350e-05,2.906232e-07,1.253398e-05,2.421720e-05,1.180346e-03,3.674943e-06,7.715760e+00
xtw_edu,4.051766e-09,1.228659e-07,6.115568e-08,3.257357e-07,5.444035e-08,5.889062e-08,1.600403e-07,1.058891e-07,1.690624e-07,3.799208e-08,...,5.054636e-05,1.350930e-04,3.803231e-05,3.463010e-05,8.131397e-05,1.730782e-05,3.435209e-05,1.630989e-03,5.631376e-06,4.438348e+00
xtw_hht,1.041254e-12,3.530940e-10,3.199517e-10,1.959990e-09,3.689138e-11,6.027289e-10,1.794476e-08,9.813307e-09,1.222204e-10,5.835578e-10,...,1.408653e-05,3.690590e-05,1.100390e-05,9.935723e-06,2.564368e-05,4.944599e-06,1.065570e-05,4.904515e-04,1.589922e-06,8.205309e+00
xtw_dwe,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,8.293493e+00


In [ ]:
df.iloc[:,:10555]

In [25]:
#df_new = df.reset_index()
df_new = df
df_new = df_new.iloc[:,0:1000]
df_new = df_new.reset_index()

# Melt the DataFrame to long format
long_df = df_new.melt(id_vars=['index'], var_name='col2', value_name='value')

# Rename the 'index' column to 'col1'
long_df = long_df.rename(columns={'index': 'col1'})

In [49]:
df_new = long_df
# Splitting 'col1' into 'col1_part1' and 'col1_part2'
df_new[['col1_part1', 'col1_part2']] = df_new['col1'].str.split('_', n=1, expand=True)

# Splitting 'col2' into 'col2_part1' and 'col2_part2'
df_new[['col2_part1', 'col2_part2']] = df_new['col2'].str.split('_', n=1, expand=True)

# Convert the new columns to uppercase
df_new[['col1_part1', 'col1_part2', 'col2_part1', 'col2_part2']] = df_new[['col1_part1', 'col1_part2', 'col2_part1', 'col2_part2']].apply(lambda x: x.str.upper())
# Drop the original 'col1' and 'col2' columns
df_new = df_new.drop(columns=['col1', 'col2'])


In [32]:
import pycountry_convert as pc

# Define a list of aggregations that should not be converted
aggregations = ['XAC', 'XCA', 'XCB', 'XEA', 'XEC', 'XEE', 
                'XEF', 'XSE', 'XNA', 'XNF', 'XOC', 'XSA',
                'XSC', 'XSE', 'XSM', 'XSU', 'XTW', 'XWF'
                'RUS', 'CHN', 'AZE', 'GEO', 'UKR', 'TUR',
                'MNG', 'UZB', 'TJK', 'KAZ', 'ARM', 'VA', 
                'TKM']

def get_continent_from_iso3(iso3):
    try:
        # Convert the ISO3 country code to ISO2 country code
        iso2 = pc.country_alpha3_to_country_alpha2(iso3)
        
        # Convert the ISO2 country code to continent code
        continent_code = pc.country_alpha2_to_continent_code(iso2)
        
        # Convert the continent code to continent name
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        
        return continent_name
    except KeyError:
        return iso3  # Return the original code if it's not a valid ISO3 code

def convert_column_to_continents(column):
    return column.apply(lambda x: get_continent_from_iso3(x) if x not in aggregations else x)

In [ ]:

# Apply the conversion to the relevant columns
df_new['col1_part1_continent'] = convert_column_to_continents(df_new['col1_part1'])
df_new['col2_part1_continent'] = convert_column_to_continents(df_new['col2_part1'])

#### OTHERS
df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XOC', 'Oceania')
df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XOC', 'Oceania')

df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSA', 'Asia')
df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSA', 'Asia')

df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSC', 'Africa')
df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSC', 'Africa')

df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSE', 'Asia')
df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSE', 'Asia')

df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSM', 'America')
df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSM', 'America')

df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XWF', 'Africa')
df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XWF', 'Africa')

df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XWS', 'Asia')
df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XWS', 'Asia')



In [ ]:
#### Countries

In [51]:
df_new = df_new.drop(columns=['col1_part1', 'col2_part1'])

In [32]:
df_new

,value,col1_part2,col2_part2,col1_part1_continent,col2_part1_continent
0,3.563641e-01,PDR,PDR,Oceania,Oceania
1,1.761312e+00,WHT,PDR,Oceania,Oceania
2,9.335614e-01,GRO,PDR,Oceania,Oceania
3,6.943234e-03,V_F,PDR,Oceania,Oceania
4,1.879592e+00,OSD,PDR,Oceania,Oceania
...,...,...,...,...,...
10400995,7.767942e-08,OSG,OFD,XTW,Asia
10400996,7.812599e-07,EDU,OFD,XTW,Asia
10400997,3.012990e-08,HHT,OFD,XTW,Asia
10400998,0.000000e+00,DWE,OFD,XTW,Asia


In [52]:
grouped_df = df_new.groupby(['col1_part2', 'col2_part2', 'col1_part1_continent', 'col2_part1_continent']).sum().reset_index()

In [ ]:
### Try to put into a function

In [33]:
def divide_agg(df_new):
    # Apply the conversion to the relevant columns
    df_new['col1_part1_continent'] = convert_column_to_continents(df_new['col1_part1'])
    df_new['col2_part1_continent'] = convert_column_to_continents(df_new['col2_part1'])

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XOC', 'Oceania')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XOC', 'Oceania')
    
    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSA', 'Asia')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSA', 'Asia')
    
    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSC', 'Africa')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSC', 'Africa')
    
    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSE', 'Asia')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSE', 'Asia')
    
    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSM', 'America')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSM', 'America')
    
    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XWF', 'Africa')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XWF', 'Africa')
    
    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XWS', 'Asia')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XWS', 'Asia')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XSU', 'Europe')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XSU', 'Europe')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XNF', 'Africa')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XNF', 'Africa')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XNA', 'America')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XNA', 'America')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XEC', 'Africa')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XEC', 'Africa')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XEF', 'Europe')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XEF', 'Europe')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XER', 'Europe')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XER', 'Europe')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XCA', 'America')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XCA', 'America')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XCB', 'America')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XCB', 'America')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XEE', 'America')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XEE', 'America')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XEA', 'Asia')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XEA', 'Asia')

    df_new['col1_part1_continent'] = df_new['col1_part1_continent'].replace('XAC', 'Africa')
    df_new['col2_part1_continent'] = df_new['col2_part1_continent'].replace('XAC', 'Africa')

    df_new = df_new.drop(columns=['col1_part1', 'col2_part1'])
    df_new = df_new.fillna('Miss')
    grouped_df = df_new.groupby(['col1_part2', 'col2_part2', 'col1_part1_continent', 'col2_part1_continent'], dropna=False).sum().reset_index()

    return(grouped_df)

In [ ]:
####

In [151]:
new_data = pd.DataFrame(columns=["col1_part2","col2_part2","col1_part1_continent","col2_part1_continent","value"])

for i in range(11):

    df_new = df
    print(i)
    
    df_new = df_new.iloc[ : , (1 + 1000*i) : ((i+1)*1000)]
    df_new = df_new.reset_index()
    
    # Melt the DataFrame to long format
    long_df = df_new.melt(id_vars=['index'], var_name='col2', value_name='value')
    
    # Rename the 'index' column to 'col1'
    long_df = long_df.rename(columns={'index': 'col1'})

    df_new = long_df
    # Splitting 'col1' into 'col1_part1' and 'col1_part2'
    df_new[['col1_part1', 'col1_part2']] = df_new['col1'].str.split('_', n=1, expand=True)
    
    # Splitting 'col2' into 'col2_part1' and 'col2_part2'
    df_new[['col2_part1', 'col2_part2']] = df_new['col2'].str.split('_', n=1, expand=True)
    
    # Convert the new columns to uppercase
    df_new[['col1_part1', 'col1_part2', 'col2_part1', 'col2_part2']] = df_new[['col1_part1', 'col1_part2', 'col2_part1', 'col2_part2']].apply(lambda x: x.str.upper())
    
    # Drop the original 'col1' and 'col2' columns
    df_new = df_new.drop(columns=['col1', 'col2'])
    

    new_ds = divide_agg(df_new) 

    new_data = pd.concat([new_data, new_ds], ignore_index=True)
    
    print(new_data['col1_part1_continent'].unique())
    
    


0


C:\Users\dadel\AppData\Local\Temp\ipykernel_4052\3208316443.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_data = pd.concat([new_data, new_ds], ignore_index=True)


['ARM' 'AZE' 'Africa' 'America' 'Asia' 'CHN' 'Europe' 'GEO' 'KAZ' 'MNG'
 'North America' 'Oceania' 'South America' 'TJK' 'TUR' 'UKR' 'UZB' 'XTW'
 'VA']
1
['ARM' 'AZE' 'Africa' 'America' 'Asia' 'CHN' 'Europe' 'GEO' 'KAZ' 'MNG'
 'North America' 'Oceania' 'South America' 'TJK' 'TUR' 'UKR' 'UZB' 'XTW'
 'VA']
2
['ARM' 'AZE' 'Africa' 'America' 'Asia' 'CHN' 'Europe' 'GEO' 'KAZ' 'MNG'
 'North America' 'Oceania' 'South America' 'TJK' 'TUR' 'UKR' 'UZB' 'XTW'
 'VA']
3
['ARM' 'AZE' 'Africa' 'America' 'Asia' 'CHN' 'Europe' 'GEO' 'KAZ' 'MNG'
 'North America' 'Oceania' 'South America' 'TJK' 'TUR' 'UKR' 'UZB' 'XTW'
 'VA']
4
['ARM' 'AZE' 'Africa' 'America' 'Asia' 'CHN' 'Europe' 'GEO' 'KAZ' 'MNG'
 'North America' 'Oceania' 'South America' 'TJK' 'TUR' 'UKR' 'UZB' 'XTW'
 'VA']
5
['ARM' 'AZE' 'Africa' 'America' 'Asia' 'CHN' 'Europe' 'GEO' 'KAZ' 'MNG'
 'North America' 'Oceania' 'South America' 'TJK' 'TUR' 'UKR' 'UZB' 'XTW'
 'VA']
6
['ARM' 'AZE' 'Africa' 'America' 'Asia' 'CHN' 'Europe' 'GEO' 'KAZ' 'MNG'
 'No

In [ ]:
##############################

In [158]:
new_data.to_csv("gtap.csv")

In [165]:
new_data

,col1_part2,col2_part2,col1_part1_continent,col2_part1_continent,value
0,AFS,AFS,ARM,Asia,0.484388
1,AFS,AFS,ARM,CHN,0.213003
2,AFS,AFS,ARM,MNG,0.006991
3,AFS,AFS,ARM,Oceania,0.671493
4,AFS,AFS,AZE,Asia,3.392482
...,...,...,...,...,...
2681585,WTR,ZWE,TJK,Y,0.000001
2681586,WTR,ZWE,TUR,Y,0.00025
2681587,WTR,ZWE,UKR,Y,0.000016
2681588,WTR,ZWE,UZB,Y,0.000018


In [44]:
### Let's adapt the data to what we need

In [45]:
# Sector Table

In [166]:
### Fix FD issue
# Define a list of aggregations that should not be converted
aggregations = ['AFS', 'ATP', 'BPH', 'B_T', 'CHM', 'CMN', 'CMT', 'CNS', 'COA',
                'CTL', 'C_B', 'DWE', 'EDU', 'EEQ', 'ELE', 'ELY', 'FMP', 'FRS',
                'FSH', 'GAS', 'GDT', 'GRO', 'HHT', 'INS', 'I_S', 'LEA', 'LUM',
                'MIL', 'MVH', 'NFM', 'NMM', 'OAP', 'OBS', 'OCR', 'OFD', 'OFI',
               'OIL', 'OME', 'OMF', 'OMT', 'OSD', 'OSG', 'OTN', 'OTP', 'OXT',
               'PCR', 'PDR', 'PFB', 'PPP', 'P_C', 'RMK', 'ROS', 'RPP', 'RSA',
               'SGR', 'TEX', 'TRD', 'VOL', 'V_F', 'WAP', 'WHS', 'WHT', 'WOL',
               'WTP', 'WTR', 'XAC', 'XCA', 'XCB', 'XEA', 'XEC', 'XEE', 
               'XEF', 'XSE', 'XNA', 'XNF', 'XOC', 'XSA',
                'XSC', 'XSE', 'XSM', 'XSU', 'XTW', 'XWF'
                'RUS', 'CHN', 'AZE', 'GEO', 'UKR', 'TUR',
                'MNG', 'UZB', 'TJK', 'KAZ', 'ARM', 'VA', 'TKM']

def get_continent_from_iso3(iso3):
    try:
        # Convert the ISO3 country code to ISO2 country code
        iso2 = pc.country_alpha3_to_country_alpha2(iso3)
        
        # Convert the ISO2 country code to continent code
        continent_code = pc.country_alpha2_to_continent_code(iso2)
        
        # Convert the continent code to continent name
        continent_name = pc.convert_continent_code_to_continent_name(continent_code)
        
        return continent_name
    except KeyError:
        return iso3  # Return the original code if it's not a valid ISO3 code

def convert_column_to_continents(column):
    return column.apply(lambda x: get_continent_from_iso3(x) if x not in aggregations else x)

In [167]:
clone_data=new_data

In [168]:
clone_data["col2_part2"] = convert_column_to_continents(clone_data["col2_part2"])

In [169]:
df_new = clone_data 

df_new["col2_part2"] = df_new["col2_part2"].replace('XOC', 'Oceania')

df_new["col2_part2"] = df_new["col2_part2"].replace('XSA', 'Asia')

df_new["col2_part2"] = df_new["col2_part2"].replace('XSC', 'Africa')

df_new["col2_part2"] = df_new["col2_part2"].replace('XSE', 'Asia')

df_new["col2_part2"] = df_new["col2_part2"].replace('XSM', 'America')

df_new["col2_part2"] = df_new["col2_part2"].replace('XWF', 'Africa')

df_new["col2_part2"] = df_new["col2_part2"].replace('XWS', 'Asia')

df_new["col2_part2"] = df_new["col2_part2"].replace('XSU', 'Europe')

df_new["col2_part2"] = df_new["col2_part2"].replace('XNF', 'Africa')

df_new["col2_part2"] = df_new["col2_part2"].replace('XNA', 'America')

df_new["col2_part2"] = df_new["col2_part2"].replace('XEC', 'Africa')

df_new["col2_part2"] = df_new["col2_part2"].replace('XEF', 'Europe')

df_new["col2_part2"] = df_new["col2_part2"].replace('XER', 'Europe')

df_new["col2_part2"] = df_new["col2_part2"].replace('XCA', 'America')

df_new["col2_part2"] = df_new["col2_part2"].replace('XCB', 'America')

df_new["col2_part2"] = df_new["col2_part2"].replace('XEE', 'America')

df_new["col2_part2"] = df_new["col2_part2"].replace('XEA', 'Asia')

df_new["col2_part2"] = df_new["col2_part2"].replace('XAC', 'Africa')
###

df_new["col2_part2"] = df_new["col2_part2"].replace('XTW', 'ROW')
df_new["col1_part1_continent"] = df_new["col1_part1_continent"].replace('XTW', 'ROW')
df_new["col2_part1_continent"] = df_new["col2_part1_continent"].replace('XTW', 'ROW')

In [170]:
df_new = df_new.groupby(['col1_part2', 'col2_part2', 'col1_part1_continent', 'col2_part1_continent'], dropna=False).sum().reset_index()

In [174]:
df_new["col2_part1_continent"].unique()

array(['ARM', 'AZE', 'Africa', 'America', 'Asia', 'CHN', 'Europe', 'GEO',
       'KAZ', 'MNG', 'North America', 'Oceania', 'ROW', 'South America',
       'TJK', 'TUR', 'UKR', 'UZB', 'Y'], dtype=object)

In [175]:
df_new

,col1_part2,col2_part2,col1_part1_continent,col2_part1_continent,value
0,AFS,AFS,ARM,ARM,13.811056
1,AFS,AFS,ARM,AZE,0.298051
2,AFS,AFS,ARM,Africa,0.214193
3,AFS,AFS,ARM,America,0.023989
4,AFS,AFS,ARM,Asia,1.853186
...,...,...,...,...,...
1391143,WTR,WTR,UZB,South America,0.004862
1391144,WTR,WTR,UZB,TJK,0.000001
1391145,WTR,WTR,UZB,TUR,0.000175
1391146,WTR,WTR,UZB,UKR,0.000058


In [ ]:
#### 

In [176]:
df_new = df_new.rename(columns={
    'col1_part2': 'from_sect',
    'col2_part2': 'to_sect',
    'col1_part1_continent': 'from_country',
    'col2_part1_continent': 'to_country'
})

In [177]:
df_new['value'] = pd.to_numeric(df_new['value'], errors='coerce')

In [122]:
## Production

In [178]:
tot_prod = df_new.groupby(['from_sect', 'from_country'], dropna=False)['value'].sum().reset_index()

In [179]:
tot_prod['sect'] = tot_prod['from_country'] + '_' + tot_prod['from_sect']
tot_prod = tot_prod.drop(columns=['from_sect', 'from_country'])
tot_prod = tot_prod[['sect', 'value']]

In [180]:
tot_prod

,sect,value
0,ARM_AFS,994.277002
1,AZE_AFS,1173.593513
2,Africa_AFS,101404.020871
3,America_AFS,26794.563152
4,Asia_AFS,946661.639780
...,...,...
1166,South America_WTR,53957.954249
1167,TJK_WTR,23.517460
1168,TUR_WTR,10650.087400
1169,UKR_WTR,1661.937957


In [134]:
### Fin Demand

In [288]:
fin_demand = df_new[df_new['to_country'] == 'Y']

In [289]:
fin_demand["from_country"].unique()

array(['ARM', 'AZE', 'Africa', 'America', 'Asia', 'CHN', 'Europe', 'GEO',
       'KAZ', 'MNG', 'North America', 'Oceania', 'ROW', 'South America',
       'TJK', 'TUR', 'UKR', 'UZB', 'VA'], dtype=object)

In [290]:
fin_demand = fin_demand.groupby(['from_sect', 'from_country'], dropna=False)['value'].sum().reset_index()
fin_demand['sect'] = fin_demand['from_country'] + '_' + fin_demand['from_sect']
fin_demand = fin_demand.drop(columns=['from_sect', 'from_country'])
fin_demand = fin_demand[['sect', 'value']]

In [291]:
fin_demand

,sect,value
0,ARM_AFS,516.338400
1,AZE_AFS,626.772030
2,Africa_AFS,61611.888007
3,America_AFS,14266.555679
4,Asia_AFS,669705.489861
...,...,...
1166,South America_WTR,24689.074081
1167,TJK_WTR,4.501616
1168,TUR_WTR,3984.706450
1169,UKR_WTR,646.184132


In [139]:
## Share of exporting firms

In [292]:
exp = df_new[df_new['to_country'] != df_new['from_country']]
exp = exp[exp['to_sect'] != exp['from_country']]

In [293]:
exp = exp.groupby(['from_sect', 'from_country'], dropna=False)['value'].sum().reset_index()
exp['sect'] = exp['from_country'] + '_' + exp['from_sect']
exp = exp.drop(columns=['from_sect', 'from_country'])
exp = exp[['sect', 'value']]

In [ ]:
### Together

In [294]:
merged_df = pd.merge(exp, tot_prod, on='sect', suffixes=('_exp', '_prod'))
merged_df = pd.merge(merged_df, fin_demand, on='sect', suffixes=('', '_find'))
merged_df = merged_df.rename(columns={'value': 'find'})

In [295]:
merged_df["share_exporting_firms"]=merged_df["value_exp"]/merged_df["value_prod"]

In [296]:
merged_df

,sect,value_exp,value_prod,find,share_exporting_firms
0,ARM_AFS,621.820543,994.277002,516.338400,0.625400
1,AZE_AFS,987.696820,1173.593513,626.772030,0.841600
2,Africa_AFS,33191.907727,101404.020871,61611.888007,0.327323
3,America_AFS,18681.799082,26794.563152,14266.555679,0.697223
4,Asia_AFS,162396.762662,946661.639780,669705.489861,0.171547
...,...,...,...,...,...
1166,South America_WTR,22.541925,53957.954249,24689.074081,0.000418
1167,TJK_WTR,0.069815,23.517460,4.501616,0.002969
1168,TUR_WTR,14.641779,10650.087400,3984.706450,0.001375
1169,UKR_WTR,0.889866,1661.937957,646.184132,0.000535


In [297]:
merged_df["usd_per_ton"] = 1000

In [298]:
merged_df["supply_data"] = "USD"

In [299]:
merged_df["cutoff"] = 0

In [300]:
## Conversion

In [301]:
code_to_category = {
    'PDR': 'agriculture',
    'WHT': 'agriculture',
    'GRO': 'agriculture',
    'V_F': 'agriculture',
    'OSD': 'agriculture',
    'C_B': 'agriculture',
    'PFB': 'agriculture',
    'OCR': 'agriculture',
    'CTL': 'agriculture',
    'OAP': 'agriculture',
    'RMK': 'agriculture',
    'WOL': 'agriculture',
    'FRS': 'agriculture',
    'FSH': 'agriculture',
    'COA': 'mining',
    'OIL': 'mining',
    'GAS': 'mining',
    'OXT': 'mining',
    'CMT': 'manufacturing',
    'OMT': 'manufacturing',
    'VOL': 'manufacturing',
    'MIL': 'manufacturing',
    'PCR': 'manufacturing',
    'SGR': 'manufacturing',
    'OFD': 'manufacturing',
    'B_T': 'manufacturing',
    'TEX': 'manufacturing',
    'WAP': 'manufacturing',
    'LEA': 'manufacturing',
    'LUM': 'manufacturing',
    'PPP': 'manufacturing',
    'P_C': 'manufacturing',
    'CHM': 'manufacturing',
    'BPH': 'manufacturing',
    'RPP': 'manufacturing',
    'NMM': 'manufacturing',
    'I_S': 'manufacturing',
    'NFM': 'manufacturing',
    'FMP': 'manufacturing',
    'ELE': 'manufacturing',
    'EEQ': 'manufacturing',
    'OME': 'manufacturing',
    'MVH': 'manufacturing',
    'OTN': 'manufacturing',
    'OMF': 'manufacturing',
    'ELY': 'utility',
    'GDT': 'utility',
    'WTR': 'utility',
    'CNS': 'service',
    'TRD': 'service',
    'AFS': 'service',
    'OTP': 'transport',
    'WTP': 'transport',
    'ATP': 'transport',
    'WHS': 'service',
    'CMN': 'service',
    'OFI': 'service',
    'ISR': 'service',
    'OBS': 'service',
    'ROS': 'service',
    'OSG': 'service',
    'DWE': 'service',
    'GOV': 'service',
    'EDU': 'service',
    'HTS': 'service',
    'RSA': 'service',
    'OCA': 'service',
    'LKS': 'service',
}

In [302]:
merged_df['sect_code'] = merged_df['sect'].str.split('_').str[1]
merged_df['type'] = merged_df['sect_code'].map(code_to_category)

In [303]:
 merged_df[merged_df['sect']=="ROW_AFS"]

,sect,value_exp,value_prod,find,share_exporting_firms,usd_per_ton,supply_data,cutoff,sect_code,type
12,ROW_AFS,1.735435,4.171546,2.816297,0.416017,1000,USD,0,AFS,service


In [304]:
# Step 1: Filter rows where sect starts with 'ROW_'
row_df = merged_df[merged_df['sect'].str.startswith('ROW_')]

# Step 2: Sum the value_prod column
row_sum = row_df['value_prod'].sum()

# Step 3: Create a new entry with sect as 'IMP'
new_entry = {
    'sect': 'IMP',
    'value_exp': 0,
    'value_prod': row_sum,
    'find': 0,
    'share_exporting_firms': 1,  # Set to 1 as required
    'usd_per_ton': 1000,  # Assuming this remains the same as other entries
    'supply_data': 'USD',  # Assuming this remains the same as other entries
    'cutoff': 0,  # Assuming this remains the same as other entries
    'sect_code': 'IMP',  # New sect_code, change if needed
    'type': 'service'  # Assuming the type, adjust if necessary
}

# Step 4: Remove the original 'ROW_' rows from the DataFrame
merged_df = merged_df[~merged_df['sect'].str.startswith('ROW_')]

# Step 5: Append the new 'IMP' entry to the DataFrame
new_entry_df = pd.DataFrame([new_entry])
merged_df = pd.concat([merged_df, new_entry_df], ignore_index=True)


In [338]:
merged_df

,sector,export_value,output,final_demand,exporting_firms_share,usd_per_ton,supply_data_currency,cutoff,sector_code,type
0,ARM_AFS,621.820543,994.277002,516.338400,0.625400,1000,USD,0,AFS,service
1,AZE_AFS,987.696820,1173.593513,626.772030,0.841600,1000,USD,0,AFS,service
2,Africa_AFS,33191.907727,101404.020871,61611.888007,0.327323,1000,USD,0,AFS,service
3,America_AFS,18681.799082,26794.563152,14266.555679,0.697223,1000,USD,0,AFS,service
4,Asia_AFS,162396.762662,946661.639780,669705.489861,0.171547,1000,USD,0,AFS,service
...,...,...,...,...,...,...,...,...,...,...
1102,TJK_WTR,0.069815,23.517460,4.501616,0.002969,1000,USD,0,WTR,utility
1103,TUR_WTR,14.641779,10650.087400,3984.706450,0.001375,1000,USD,0,WTR,utility
1104,UKR_WTR,0.889866,1661.937957,646.184132,0.000535,1000,USD,0,WTR,utility
1105,UZB_WTR,0.956193,292.362082,98.144748,0.003271,1000,USD,0,WTR,utility


In [333]:
new_column_names = {
    'sect': 'sector',
    'value_exp': 'export_value',
    'value_prod': 'output',
    'find': 'final_demand',
    'share_exporting_firms': 'exporting_firms_share',
    'usd_per_ton': 'usd_per_ton',
    'supply_data': 'supply_data_currency',
    'cutoff': 'cutoff',
    'sect_code': 'sector_code',
    'type': 'type'
}

# Use the rename method to rename the columns
merged_df = merged_df.rename(columns=new_column_names)

In [341]:
merged_df=merged_df[merged_df["sector"]!="VA_Miss"]

In [342]:
merged_df.to_csv("sector_table.csv")

# Inventories

In [311]:
import itertools

In [314]:
unique_sectors = merged_df['sect'].unique()

# Step 2: Generate all possible combinations of sectors
sector_combinations = list(itertools.product(unique_sectors, repeat=2))

# Step 3: Create a DataFrame from the combinations
combinations_df = pd.DataFrame(sector_combinations, columns=['input_sector', 'buying_sector'])

# Step 4: Initialize the inventory_duration_target column (with placeholder values, e.g., None or 0)
combinations_df['inventory_duration_target'] = 10

In [322]:
combinations_df.to_csv("inventories.csv")

# Location

In [ ]:
## ['ARM', 'AZE', 'Africa', 'America', 'Asia', 'CHN', 'Europe', 'GEO',
#       'KAZ', 'MNG', 'North America', 'Oceania', 'ROW', 'South America',
#       'TJK', 'TUR', 'UKR', 'UZB']##

In [319]:
import geojson

In [364]:
# Define the coordinates for each region
coordinates = {
    'ARM': (40.1792, 44.4991),  # Yerevan, Armenia
    'AZE': (40.4093, 49.8671),  # Baku, Azerbaijan
    'Africa': (-33.9249, 18.4241),  # Abuja, Nigeria (random choice for Africa)
    'America': (40.7128, -74.0060),  # Washington D.C., USA (random choice for America)
    'Asia':  (137.66, 37.4609),  # New Delhi, India (random choice for Asia)
    'CHN': (39.9042, 116.4074),  # Beijing, China
    'Europe': (48.8566, 2.3522),  # Paris, France (random choice for Europe)
    'GEO': (41.7151, 44.8271),  # Tbilisi, Georgia
    'KAZ': (51.1694, 71.4491),  # Nur-Sultan, Kazakhstan
    'MNG': (47.8864, 106.9057),  # Ulaanbaatar, Mongolia
    'North America': (40.7128, -74.0060),  # Mexico City, Mexico (random choice for North America)
    'Oceania': (-41.2865, 174.7762),  # Canberra, Australia (random choice for Oceania)
    'ROW': (-18.7669, 46.8691),  # Antananarivo, Madagascar
    'South America': (38.9072, -77.0369),  # Buenos Aires, Argentina (random choice for South America)
    'TJK': (38.5598, 68.7870),  # Dushanbe, Tajikistan
    'TUR': (39.9334, 32.8597),  # Ankara, Turkey
    'UKR': (50.4501, 30.5234),  # Kyiv, Ukraine
    'UZB': (41.2995, 69.2401)   # Tashkent, Uzbekistan
}

# Create the GeoJSON features
features = []
for region, (lat, lon) in coordinates.items():
    feature = geojson.Feature(
        geometry=geojson.Point((lon, lat)),
        properties={"admin_code": region}
    )
    features.append(feature)

# Create the GeoJSON FeatureCollection
feature_collection = geojson.FeatureCollection(features)

# Save to a GeoJSON file
geojson_path = 'region_table_good.geojson'
with open(geojson_path, 'w') as f:
    geojson.dump(feature_collection, f)

In [348]:
feature_collection

{"features": [{"geometry": {"coordinates": [44.4991, 40.1792], "type": "Point"}, "properties": {"admin_code": "ARM"}, "type": "Feature"}, {"geometry": {"coordinates": [49.8671, 40.4093], "type": "Point"}, "properties": {"admin_code": "AZE"}, "type": "Feature"}, {"geometry": {"coordinates": [8.6753, 9.082], "type": "Point"}, "properties": {"admin_code": "Africa"}, "type": "Feature"}, {"geometry": {"coordinates": [-77.0369, 38.9072], "type": "Point"}, "properties": {"admin_code": "America"}, "type": "Feature"}, {"geometry": {"coordinates": [77.209, 28.6139], "type": "Point"}, "properties": {"admin_code": "Asia"}, "type": "Feature"}, {"geometry": {"coordinates": [116.4074, 39.9042], "type": "Point"}, "properties": {"admin_code": "CHN"}, "type": "Feature"}, {"geometry": {"coordinates": [2.3522, 48.8566], "type": "Point"}, "properties": {"admin_code": "Europe"}, "type": "Feature"}, {"geometry": {"coordinates": [44.8271, 41.7151], "type": "Point"}, "properties": {"admin_code": "GEO"}, "type"

# MRIO TABLE

In [349]:
df_new

,from_sect,to_sect,from_country,to_country,value,from
0,AFS,AFS,ARM,ARM,13.811056,ARM_AFS
1,AFS,AFS,ARM,AZE,0.298051,ARM_AFS
2,AFS,AFS,ARM,Africa,0.214193,ARM_AFS
3,AFS,AFS,ARM,America,0.023989,ARM_AFS
4,AFS,AFS,ARM,Asia,1.853186,ARM_AFS
...,...,...,...,...,...,...
1391143,WTR,WTR,UZB,South America,0.004862,UZB_WTR
1391144,WTR,WTR,UZB,TJK,0.000001,UZB_WTR
1391145,WTR,WTR,UZB,TUR,0.000175,UZB_WTR
1391146,WTR,WTR,UZB,UKR,0.000058,UZB_WTR


In [64]:
### Codense the ROW nodes

In [350]:
mrio = df_new
mrio['from'] = mrio['from_country'] + '_' + mrio['from_sect']
mrio = mrio.drop(columns=['from_sect', 'from_country'])
mrio

,to_sect,to_country,value,from
0,AFS,ARM,13.811056,ARM_AFS
1,AFS,AZE,0.298051,ARM_AFS
2,AFS,Africa,0.214193,ARM_AFS
3,AFS,America,0.023989,ARM_AFS
4,AFS,Asia,1.853186,ARM_AFS
...,...,...,...,...
1391143,WTR,South America,0.004862,UZB_WTR
1391144,WTR,TJK,0.000001,UZB_WTR
1391145,WTR,TUR,0.000175,UZB_WTR
1391146,WTR,UKR,0.000058,UZB_WTR


In [351]:
mrio.loc[mrio['to_country'] == 'Y', 'to_country'] = 'final_demand'
mrio.loc[mrio['to_country'] == 'final_demand', ['to_sect', 'to_country']] = mrio.loc[mrio['to_country'] == 'final_demand', ['to_country', 'to_sect']].values
mrio['to'] = mrio['to_country'] + '_' + mrio['to_sect']
mrio = mrio.drop(columns=['to_sect', 'to_country'])
mrio

,value,from,to
0,13.811056,ARM_AFS,ARM_AFS
1,0.298051,ARM_AFS,AZE_AFS
2,0.214193,ARM_AFS,Africa_AFS
3,0.023989,ARM_AFS,America_AFS
4,1.853186,ARM_AFS,Asia_AFS
...,...,...,...
1391143,0.004862,UZB_WTR,South America_WTR
1391144,0.000001,UZB_WTR,TJK_WTR
1391145,0.000175,UZB_WTR,TUR_WTR
1391146,0.000058,UZB_WTR,UKR_WTR


In [202]:
### Put together Importer and Exporter

In [352]:
prefixes_to_change = ["Asia", "Africa", "America", "ROW", "North America",
                     "Europe","Oceania","South America"]  # Add other prefixes as needed

# Function to modify the 'from' column
def modify_from(row):
    parts = row.split('_')
    if parts[0] in prefixes_to_change:
        return f"{parts[0]}_Imports"
    return row

# Function to modify the 'to' column
def modify_to(row):
    parts = row.split('_')
    if parts[0] in prefixes_to_change:
        return f"{parts[0]}_Exports"
    return row

# Apply the modification to the 'from' and 'to' columns
mrio['from'] = mrio['from'].apply(modify_from)
mrio['to'] = mrio['to'].apply(modify_to)

In [353]:
mrio = mrio.groupby(['from', 'to'], dropna=False)['value'].sum().reset_index()

In [244]:
mrio

,from,to,value
0,ARM_AFS,ARM_AFS,13.811056
1,ARM_AFS,ARM_ATP,1.081747
2,ARM_AFS,ARM_BPH,0.010089
3,ARM_AFS,ARM_B_T,0.638056
4,ARM_AFS,ARM_CHM,0.114468
...,...,...,...
440207,VA_Miss,UZB_WHT,560.196632
440208,VA_Miss,UZB_WOL,53.483971
440209,VA_Miss,UZB_WTP,51.418777
440210,VA_Miss,UZB_WTR,93.747260


In [222]:
## Pivot

In [354]:
df = mrio

# Step 1: Ensure the 'from' and 'to' columns are strings
df['from'] = df['from'].astype(str)
df['to'] = df['to'].astype(str)

# Step 2: Split only on the first underscore to separate country and sector
df['from_country'] = df['from'].str.extract(r'^([^_]+)')
df['from_sector'] = df['from'].str.extract(r'^[^_]+_(.*)')

df['to_country'] = df['to'].str.extract(r'^([^_]+)')
df['to_sector'] = df['to'].str.extract(r'^[^_]+_(.*)')

# Step 3: Create the pivot table with multi-index rows and columns
pivot_df = df.pivot_table(
    index=['from_country', 'from_sector'],
    columns=['to_country', 'to_sector'],
    values='value',
    aggfunc='sum',
    fill_value=0
)

# Step 4: Reset the index to remove hierarchical indexing
pivot_df = pivot_df.reset_index()

# Optional: Set the new index to include the multi-index
pivot_df = pivot_df.set_index(['from_country', 'from_sector'])

# Step 5: Reorder the columns if needed
pivot_df = pivot_df.sort_index(axis=1)

pivot_df = pivot_df.drop(pivot_df.index[-1])

In [355]:
pivot_df.to_csv("mrio.csv")